# Inspeksjon av Collection-datasettet

In [ ]:
#Notebook config
%load_ext autoreload
%autoreload 2

figsize=(14, 4)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
data_folder = os.path.join('../../..', 'data/prod')
file_name = "Collection_data.csv"
file_path = os.path.join(data_folder, file_name)

In [ ]:
df = pd.read_csv(file_path, sep=';', decimal=',')

### Datasettet
Datasettet består av kolonnene
- `yearmonth`: År og måned i formatet ÅÅÅÅMM
- `collectionid`: ID for inkassosaken
- `PersonId`: ID for person
- `registreringsmaaned`: registreringsperiode for når sak ble sendt til inkasso i formatet ÅÅÅÅMM
- `AccountId`: ID for konto
- `producttype`: Produkttype
- `productname`: Produktnavn
- `CollectionOpenedDateId`: Dato for når sak ble sendt til inkasso
- `CollectionClosedDateId`: Dato for når inkassosak ble lukket
- `BalanceSentAmt`: Balanse som ble sendt til inkasso (Hovedstol)
- `PaymentAmt`: Innbetalinger til Kredittbanken direkte  i perioden
- `PaymentRTAmt`: Sum innbetalt til Kredittbanken direkte på sak totalt 
- `PrincipalPaymentAmt`: Innbetalinger via inkassoselskapet i perioden
- `PrincipalPaymentRTAmt`: Sum innbetalinger gjort til inkassoselskapet totalt
- `LossAmt`: Tapsført i perioden
- `LossRTAmt`: Totalt tapsførst på sak
- `MonthInDCA`: Antall måneder sak har ligget hos inkasso
- `MonthsInZCOV`: Antall måneder sak har ligget til overvåk (tapsført)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df = df.sort_values(by=["PersonId", "yearmonth"], ascending=True)
df[df['PersonId'] == 214].head(100)